In [2]:
import lib.io_ as io
import pandas as pd

In [5]:
%%time
df = io.load_path_file('random_file_paths.txt', drop_subset = ['recording_MBID', 'artist_MBID'])
df.drop(['release_MBID'], axis = 1, inplace = True)
df.drop_duplicates(['artist_MBID', 'recording_MBID'], inplace = True)
df.rename({'artist_MBID': 'mlhd_artist_mbid', 'recording_MBID': 'mlhd_recording_mbid'}, inplace=True, axis=1)
df.reset_index(inplace=True, drop=True)

df.head()

CPU times: user 5.48 s, sys: 490 ms, total: 5.97 s
Wall time: 5.98 s


,timestamp,mlhd_artist_mbid,mlhd_recording_mbid
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,1deb956c-5439-4fbb-b026-5adb4330a934
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,14e9eb4e-155d-46ff-9a83-a8d5e1936c81
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,eced9a9b-cd59-40f8-a580-f27094bd8a89
3,1108594566,e6e879c0-3d56-4f12-b3c5-3ce459661a8e,0a8e9fce-b54c-45dd-8081-4aaa654ef4ec
4,1108679759,3c0a0074-4f26-4d3b-b723-a66bf6cc3753,06a5b648-0950-4892-a975-715291a5de6f


In [6]:
%%time
# MB_rec_names = mb.get_recording_name()
# MB_rec_names = pd.read_parquet('warehouse/MB_tables/MB_rec_names.parquet')
# MB_rec_names.set_index('gid', inplace=True)

# MB_rec_redir = mb.get_recording_redirects()
MB_rec_redirects = pd.read_parquet('warehouse/MB_tables/MB_rec_redirects.parquet')
MB_rec_redirects.set_index('old', inplace=True)

# MB_rec_canonical = mb.get_recording_canonical()
MB_rec_canonical = pd.read_parquet('warehouse/MB_tables/MB_rec_canonical.parquet')
MB_rec_canonical.set_index('old', inplace=True)

MB_artist_credit_list = pd.read_parquet('warehouse/MB_tables/MB_artist_credit.parquet')
MB_artist_credit_list.set_index('rec_gid', inplace=True)

CPU times: user 23.6 s, sys: 8.87 s, total: 32.5 s
Wall time: 28.2 s


In [8]:
%%time
### mlhd_recording_mbid ###

df['mlhd_canonical_mbid'] = df['mlhd_recording_mbid'].map(lambda x: io.replace(x, MB_rec_redirects, 'new'))
df['mlhd_canonical_mbid'] = df['mlhd_canonical_mbid'].map(lambda x: io.replace(x, MB_rec_canonical, 'new'))
# df['recording_name'] = df['mlhd_canonical_mbid'].map(lambda x: replace(x, MB_rec_names, 'name'))

rec_name_artist_credit = df['mlhd_canonical_mbid'].map(lambda x: io.replace_multi(x, MB_artist_credit_list))
df['rec_name'], df['artist_credit'] = zip(*rec_name_artist_credit)

df.dropna(subset = ['rec_name', 'artist_credit'], inplace=True)

df.head()

CPU times: user 8.67 s, sys: 11.5 ms, total: 8.68 s
Wall time: 8.69 s


,timestamp,mlhd_artist_mbid,mlhd_recording_mbid,mlhd_canonical_mbid,rec_name,artist_credit
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,1deb956c-5439-4fbb-b026-5adb4330a934,1deb956c-5439-4fbb-b026-5adb4330a934,Anywhere,Evanescence
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,14e9eb4e-155d-46ff-9a83-a8d5e1936c81,b0e80c50-6bca-415c-9515-1b4983329b80,Lady Lady,Mark Joseph
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,eced9a9b-cd59-40f8-a580-f27094bd8a89,6ae8e64a-d208-4f4e-8d88-155ed0568344,Building a Mystery,Sarah McLachlan
3,1108594566,e6e879c0-3d56-4f12-b3c5-3ce459661a8e,0a8e9fce-b54c-45dd-8081-4aaa654ef4ec,da1308d1-6037-4ba1-b6fc-a643a4201140,Hallelujah,Jeff Buckley
4,1108679759,3c0a0074-4f26-4d3b-b723-a66bf6cc3753,06a5b648-0950-4892-a975-715291a5de6f,a2a800b4-4969-4bc6-88e9-6928b973890c,Penny & Me,Hanson


In [9]:
df.to_csv('warehouse/mapper_outputs/mlhd_artist_credits.csv', index=False)

# MBID Mapping

In [10]:
df = pd.read_csv('warehouse/mapper_outputs/mlhd_artist_credits.csv')
test_df = df[:50]

In [11]:
import requests
import requests_cache
from urllib.parse import urlencode

requests_cache.install_cache('warehouse/mlhd_cache', expire_after=86400, allowable_methods=['GET', 'POST'])

In [12]:
# Functions

def gen_chunk(df, chunksize=15):
    """Breaks the dataframe into chunks of size "chunksize".

    Args:
        df (Pandas.DataFrame): input dataframe
        chunksize (int, optional): _description_. Defaults to 15.

    Yields:
        Pandas.DataFrame: A DataFrame of size <= "chunksize".
    """
    for i in range(0, len(df), chunksize):
        yield df[i:i+chunksize]

def make_payload(df):
    """Takes in a dataframe and returns a list of dictionaries with artist_credit_name and recording_name in required format.

    Args:
        df (Pandas.DataFrame): Input DataFrame containing columns artist_credit and rec_name

    Returns:
        list: A list of dictionaries with artist_credit_name and recording_name in required format.
    """
    payload = []
    try:
        series_artist_name = df.artist_credit.to_list()
        series_rec_name = df.rec_name.to_list()
    except AttributeError:
        print(df)
    
    for artist_name, rec_name in zip(series_artist_name, series_rec_name):
        json_inp = dict()
        json_inp["[artist_credit_name]"] = artist_name
        json_inp["[recording_name]"] = rec_name
        payload.append(json_inp)
    return payload

def get_mapping(payload, mbc = True):
    """Function to get post request reply from payload

    Args:
        payload list: a list of dictionaries with artist_credit_name and recording_name in required format
        mbc: boolean to select MBC or mbid-mapping as the request endpoint. Defaults to False.
    Returns:
        list: list of dictionaries where each dictionary is a reply for a row.
    """
    if mbc == False:
        base_url = "https://labs.api.listenbrainz.org/mbid-mapping/json"
    else:
        base_url = "https://datasets.listenbrainz.org/mbc-lookup/json"
    r = requests.post(base_url, json = payload)
    try:
        return r.json()
    except:
        return "[{}]"

def extract_mapping(replies: list, features = ['artist_credit_arg', 'recording_arg', 'recording_mbid']):
    """takes replies received from get_mapping function and extracts the required features.

    Args:
        replies (list): replies received from get_mapping function
        features (list, optional): _description_. Defaults to ['artist_credit_arg', 'recording_arg', 'recording_mbid'].

    Returns:
        list: list of tuples where each tuple is a row of the extracted features.
    """
    return list(tuple(reply[feature] for feature in features) for reply in replies)

In [13]:
# extract_mapping(get_mapping(make_payload((test_df[:100])), mbc = True))

In [14]:
def mapper(df_input, mbc = True):
    """Master Function to fetch and map replies from mbid-mapper to input dataframe"""
    replies = []
    for chunk in gen_chunk(df_input, 15):
        # payload = make_payload((chunk))
        reply = extract_mapping(get_mapping(make_payload(chunk), mbc = mbc))
        replies.extend(reply)
    
    reply_df = pd.DataFrame(replies)
    reply_df.set_index([0, 1], inplace=True)
    reply_df.rename(columns={2: 'received_rec_mbid'}, inplace=True)
    
    return df_input.join(reply_df, on = ['artist_credit', 'rec_name'], how='left')

In [15]:
%%time
test_run_mapper = mapper(df.iloc[:1000, :], mbc = False)
test_run_mapper.to_csv('warehouse/mapper_outputs/mlhd_artist_credits_mbid-mapper_1000.csv', index=False)
test_run_mapper.head()

CPU times: user 1.18 s, sys: 114 ms, total: 1.3 s
Wall time: 1min 2s


,timestamp,mlhd_artist_mbid,mlhd_recording_mbid,mlhd_canonical_mbid,rec_name,artist_credit,received_rec_mbid
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,1deb956c-5439-4fbb-b026-5adb4330a934,1deb956c-5439-4fbb-b026-5adb4330a934,Anywhere,Evanescence,1deb956c-5439-4fbb-b026-5adb4330a934
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,14e9eb4e-155d-46ff-9a83-a8d5e1936c81,b0e80c50-6bca-415c-9515-1b4983329b80,Lady Lady,Mark Joseph,b0e80c50-6bca-415c-9515-1b4983329b80
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,eced9a9b-cd59-40f8-a580-f27094bd8a89,6ae8e64a-d208-4f4e-8d88-155ed0568344,Building a Mystery,Sarah McLachlan,6ae8e64a-d208-4f4e-8d88-155ed0568344
3,1108594566,e6e879c0-3d56-4f12-b3c5-3ce459661a8e,0a8e9fce-b54c-45dd-8081-4aaa654ef4ec,da1308d1-6037-4ba1-b6fc-a643a4201140,Hallelujah,Jeff Buckley,da1308d1-6037-4ba1-b6fc-a643a4201140
4,1108679759,3c0a0074-4f26-4d3b-b723-a66bf6cc3753,06a5b648-0950-4892-a975-715291a5de6f,a2a800b4-4969-4bc6-88e9-6928b973890c,Penny & Me,Hanson,a2a800b4-4969-4bc6-88e9-6928b973890c


In [16]:
%%time
test_run_mbc = mapper(df.iloc[:1000, :], mbc=True)
test_run_mbc.to_csv('warehouse/mapper_outputs/mlhd_artist_credits_mbc_1000.csv', index=False)
test_run_mbc.head()

CPU times: user 1.19 s, sys: 107 ms, total: 1.3 s
Wall time: 8.93 s


,timestamp,mlhd_artist_mbid,mlhd_recording_mbid,mlhd_canonical_mbid,rec_name,artist_credit,received_rec_mbid
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,1deb956c-5439-4fbb-b026-5adb4330a934,1deb956c-5439-4fbb-b026-5adb4330a934,Anywhere,Evanescence,1deb956c-5439-4fbb-b026-5adb4330a934
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,14e9eb4e-155d-46ff-9a83-a8d5e1936c81,b0e80c50-6bca-415c-9515-1b4983329b80,Lady Lady,Mark Joseph,b0e80c50-6bca-415c-9515-1b4983329b80
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,eced9a9b-cd59-40f8-a580-f27094bd8a89,6ae8e64a-d208-4f4e-8d88-155ed0568344,Building a Mystery,Sarah McLachlan,6ae8e64a-d208-4f4e-8d88-155ed0568344
3,1108594566,e6e879c0-3d56-4f12-b3c5-3ce459661a8e,0a8e9fce-b54c-45dd-8081-4aaa654ef4ec,da1308d1-6037-4ba1-b6fc-a643a4201140,Hallelujah,Jeff Buckley,da1308d1-6037-4ba1-b6fc-a643a4201140
4,1108679759,3c0a0074-4f26-4d3b-b723-a66bf6cc3753,06a5b648-0950-4892-a975-715291a5de6f,a2a800b4-4969-4bc6-88e9-6928b973890c,Penny & Me,Hanson,a2a800b4-4969-4bc6-88e9-6928b973890c


In [17]:
test_run_mbc.compare(test_run_mapper)

received_rec_mbid  \
                                     self   
5    5ae06fb2-05a7-447f-94cd-0e26693f87b0   
6    24918272-d403-447c-97c3-e9f115a84360   
7    661a883e-89b2-4b35-bac5-c8534ed9f39d   
8    09bcd96c-75b5-4225-8b21-29ad3e5516e0   
12   560062d0-e122-46be-9d89-0f3b44077fdf   
..                                    ...   
958  c62ebe88-30e2-49c6-8fb7-ca663acb7ef1   
975  3e70d05e-aa8d-4e5b-86d4-a2878d4bd356   
981  c3c9d901-20c4-4286-818e-3b83701f9ea2   
990  055c56fa-f2fc-4bab-999e-32e4b06a1a53   
994  b47bf632-f6ca-42fa-9db9-10d38797aa9d   

                                           
                                    other  
5                                     NaN  
6                                     NaN  
7                                     NaN  
8                                     NaN  
12                                    NaN  
..                                    ...  
958  eb55876d-d83a-491c-9707-591ee136223d  
975                                   NaN  
981                                   NaN  
990  217c4f4a-d6d1-44db-9581-4cc83b738446  
994                                   NaN  

[107 rows x 2 columns]

In [27]:
print("mbc", test_run_mbc.received_rec_mbid.isna().value_counts())
print()
print("mbid-mapper", test_run_mapper.received_rec_mbid.isna().value_counts())

mbc False    1008
True        2
Name: received_rec_mbid, dtype: int64

mbid-mapper False    940
True      70
Name: received_rec_mbid, dtype: int64


# Visualization

In [22]:
from jinja2 import Environment, BaseLoader
jenv = Environment(loader=BaseLoader)
from IPython.display import display, HTML
from datetime import datetime

In [28]:
def write_html(df, base_path='/home/snaek/public_html/', suffix='mbc'):
    template = """
<table>
<tr><th>Artist</th><th>Recording</th><th>Canonical MBID</th><th>Canonical Data Lookup</th></tr>
{% for key,value in df.iterrows() %}
<tr>
<td>{{value['artist_credit']}}</td>
<td>{{value['rec_name']}}</td>
<td><a href="https://musicbrainz.org/recording/{{value['mlhd_canonical_mbid']}}">{{value['mlhd_canonical_mbid']}}</a></td>
<td><a href="https://musicbrainz.org/recording/{{value['received_rec_mbid']}}">{{value['received_rec_mbid']}}</a> (<a href="https://datasets.listenbrainz.org/mbc-lookup?%5Bartist_credit_name%5D={{value['artist_credit']}}&%5Brecording_name%5D={{value['rec_name']}}">lookup</a>)</td>
</tr>
{% endfor %}
</table>
"""
    f_name = "mlhd-lookup-{}-{}.html".format(datetime.now().strftime("%y-%m-%d"), suffix)
    jtemplate = jenv.from_string(template)
    different = df[df.mlhd_canonical_mbid!=df.received_rec_mbid]
    report_html = jtemplate.render(df=different)
    
    print
    with open(base_path+f_name, "w", encoding="utf-8") as fp:
        fp.write(report_html) 

In [30]:
write_html(test_run_mbc, suffix='mbc')
write_html(test_run_mapper, suffix='mbid-mapper')

In [135]:
# jtemplate = jenv.from_string(template)

# different = test_run_mbc[test_run_mbc.mlhd_canonical_mbid!=test_run_mbc.received_rec_mbid]

# report_html = jtemplate.render(df=different)
# with open("/home/snaek/public_html/mlhd-lookup-2022-07-25-mbc.html", "w", encoding="utf-8") as fp:
#     fp.write(report_html)

In [134]:
# jtemplate = jenv.from_string(template)

# different = test_run_mbc[test_run_mbc.mlhd_canonical_mbid!=test_run_mbc.received_rec_mbid]

# report_html = jtemplate.render(df=different)
# with open("/home/snaek/public_html/mlhd-lookup-2022-07-25-mbid_mapper.html", "w", encoding="utf-8") as fp:
#     fp.write(report_html)